In [2]:
NUM_ROUNDS = {
    # (block_size, key_size): num_rounds
    (32, 64): 32,
    (48, 96): 36,
    (64, 128): 44,
}


def get_sequence(num_rounds):
    if num_rounds < 40:
        states = [1] * 5
    else:
        states = [1] * 6

    for i in range(num_rounds - 5):
        if num_rounds < 40:
            feedback = states[i + 2] ^ states[i]
        else:
            feedback = states[i + 1] ^ states[i]
        states.append(feedback)

    return tuple(states)


class Simeck:
    def __init__(self, block_size, key_size, master_key):
        assert (block_size, key_size) in NUM_ROUNDS
        assert 0 <= master_key < (1 << key_size)
        self._block_size = block_size
        self._key_size = key_size
        self._word_size = block_size // 2
        self._num_rounds = NUM_ROUNDS[(block_size, key_size)]
        self._sequence = get_sequence(self._num_rounds)
        self._modulus = 1 << self._word_size
        self.change_key(master_key)

    def _LROT(self, x, r):
        assert 0 <= x < self._modulus
        res = (x << r) % self._modulus
        res |= x >> (self._word_size - r)
        return res

    def _round(self, round_key, left, right):
        assert 0 <= round_key < self._modulus
        assert 0 <= left < self._modulus
        assert 0 <= right < self._modulus
        temp = left
        left = right ^ (left & self._LROT(left, 5)) \
            ^ self._LROT(left, 1) ^ round_key
        right = temp
        # print hex(round_key), hex(left), hex(right)
        return left, right

    def change_key(self, master_key):
        assert 0 <= master_key < (1 << self._key_size)
        states = []
        for i in range(self._key_size //self._word_size):
            states.append(master_key % self._modulus)
            master_key >>= self._word_size

        constant = self._modulus - 4
        round_keys = []
        for i in range(self._num_rounds):
            round_keys.append(states[0])
            left, right = states[1], states[0]
            left, right = self._round(constant ^ self._sequence[i],
                                      left, right)
            states.append(left)
            states.pop(0)
            states[0] = right

        self.__round_keys = tuple(round_keys)

    def encrypt(self, plaintext,num_rounds):
        assert 0 <= plaintext < (1 << self._block_size)
        left = plaintext >> self._word_size
        right = plaintext % self._modulus

        for idx in range(num_rounds):
            left, right = self._round(self.__round_keys[idx],
                                      left, right)

        ciphertext = (left << self._word_size) | right
        return ciphertext


def print_test_vector(block_size, key_size, key, plain, cipher):
    print ('Simeck', block_size, key_size)
    print ('key', hex(key)[2:].rstrip('L').zfill(key_size // 4))
    print ('plaintext', hex(plain)[2:].rstrip('L').zfill(block_size // 4))
    print ('ciphertext', hex(cipher)[2:].rstrip('L').zfill(block_size // 4))




In [9]:
import math
key64 = 0x191811100908010f
ciphertext32_1,ciphertext32_2,ciphertext32_3,ciphertext32_4,ciphertext32_5,ciphertext32_6,ciphertext32_7,ciphertext32_8=0,0,0,0,0,0,0,0
simeck32 = Simeck(32, 64, key64)
for i in range(2**15):
    plaintext32  = i<<17 #^ 0x19181170  
    ciphertext32_1 ^= simeck32.encrypt(plaintext32,1)
    ciphertext32_2 ^= simeck32.encrypt(plaintext32,2)
    ciphertext32_3 ^= simeck32.encrypt(plaintext32,3)
    ciphertext32_4 ^= simeck32.encrypt(plaintext32,4)
    ciphertext32_5 ^= simeck32.encrypt(plaintext32,5)
    ciphertext32_6 ^= simeck32.encrypt(plaintext32,6)
    ciphertext32_7 ^= simeck32.encrypt(plaintext32,7)
    ciphertext32_8 ^= simeck32.encrypt(plaintext32,8)
print(i,"  ",hex(ciphertext32_1),"\t",hex(ciphertext32_2),"\t",hex(ciphertext32_3),"\t",hex(ciphertext32_4),"\t",hex(ciphertext32_5),"\t",hex(ciphertext32_6),"\t",hex(ciphertext32_7),"\t",hex(ciphertext32_8))


32767    0x0 	 0x0 	 0x0 	 0x0 	 0x0 	 0xc0000 	 0x2e18000c 	 0x2f792e18
